In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [4]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

spacex_df=pd.read_csv(URL)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [8]:
for i in range(len(launch_sites_df)):
    coordinate = [launch_sites_df.loc[i,'Lat'], launch_sites_df.loc[i,'Long']]
    circle = folium.Circle(coordinate, radius = 1000, color = '%000000', fill = True).add_child(folium.Popup(launch_sites_df.loc[i,'Launch Site']))
    marker = folium.map.Marker(coordinate,
                               icon = DivIcon(icon_size=(20,20),
                               icon_anchor=(0,0),
                              html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_sites_df.loc[i,'Launch Site'], ))
    site_map.add_child(circle)
    site_map.add_child(marker)

In [9]:
site_map

In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [11]:
marker_cluster = MarkerCluster()


In [12]:
def getcolor(df, i):
  if df.loc[i, 'class'] == 1:
    return 'green'
  else:
    return 'red'

In [13]:
# Add marker_cluster to current site_map
print(spacex_df.groupby(['Launch Site']))
site_map.add_child(marker_cluster)



# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for i in range(len(spacex_df)):
    color = getcolor(spacex_df, i)

    marker = folium.Marker(location = [spacex_df.loc[i,'Lat'],
                                       spacex_df.loc[i,'Long']],
                           popup = spacex_df.loc[i,'Launch Site'],
                                       icon=folium.Icon(color=color))
    marker_cluster.add_child(marker)

site_map

In [16]:
import folium
from folium.plugins import MarkerCluster

# Create base map centered on an approximate launch location
site_map = folium.Map(location=[28.5721, -80.6480], zoom_start=6)

# Function to determine marker color based on success/failure
def get_color(success):
    return "green" if success == 1 else "red"  # Green = Success, Red = Failure

# Dictionary to store marker clusters for each launch site
site_clusters = {}

# Loop through each unique launch site
for site, site_data in spacex_df.groupby("Launch Site"):
    # Create a marker cluster for the site
    site_clusters[site] = MarkerCluster(name=site).add_to(site_map)

    # Add each launch event to the respective site cluster
    for index, record in site_data.iterrows():
        color = get_color(record["class"])  # Determine color

        marker = folium.Marker(
            location=[record["Lat"], record["Long"]],
            popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}",
            tooltip="Click for details",
            icon=folium.Icon(color=color)
        )

        site_clusters[site].add_child(marker)  # Add marker to site-specific cluster

# Display the map
site_map


In [17]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

VAFB:
Train: 34.63346, -120.6249; Coastline: 34.63591, -120.6251

KSC:
Highway: 28.5722, -80.65572

CCAFS:
Highway: 28.55492, -80.5762; Coastline, 28.56232, -80.56766

In [18]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [19]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [26]:
coastlinecali = [34.63591, -120.6251]
coastlineflor = [28.56232, -80.56766]

VAFB = calculate_distance(coastlinecali[0], coastlinecali[1], launch_sites_df.loc[3, 'Lat'], launch_sites_df.loc[3, 'Long'])
LC40 = calculate_distance(coastlineflor[0], coastlineflor[1], launch_sites_df.loc[0, 'Lat'], launch_sites_df.loc[0, 'Long'])
SLC40 = calculate_distance(coastlineflor[0], coastlineflor[1], launch_sites_df.loc[1, 'Lat'], launch_sites_df.loc[1, 'Long'])
KSC = calculate_distance(coastlineflor[0], coastlineflor[1], launch_sites_df.loc[2, 'Lat'], launch_sites_df.loc[2, 'Long'])

print(VAFB)
print(LC40)
print(SLC40)
print(KSC)

launch_sites_df['Distance'] = [VAFB, LC40, SLC40, KSC]
launch_sites_df

1.3575337365146314
0.9472794037349772
0.9001694267424337
7.8352956774005955


,Launch Site,Lat,Long,Distance
0,CCAFS LC-40,28.562302,-80.577356,1.357534
1,CCAFS SLC-40,28.563197,-80.576820,0.947279
2,KSC LC-39A,28.573255,-80.646895,0.900169
3,VAFB SLC-4E,34.632834,-120.610745,7.835296


In [29]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
# for example
distance_marker1 = folium.Marker(
    location = [coastlinecali[0], coastlinecali[1]],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(VAFB),
        )
    )
site_map.add_child(distance_marker1)

distance_marker2 = folium.Marker(
    location = [coastlineflor[0], coastlineflor[1]],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(LC40),
        )
    )
site_map.add_child(distance_marker2)

In [32]:
lines = folium.PolyLine(locations=[[coastlinecali[0], coastlinecali[1]], [launch_sites_df.loc[3, 'Lat'], launch_sites_df.loc[3, 'Long']]], weight=1)
site_map.add_child(lines)

lines2 = folium.PolyLine(locations=[[coastlineflor[0], coastlineflor[1]], [launch_sites_df.loc[0, 'Lat'], launch_sites_df.loc[0, 'Long']]], weight=1)
site_map.add_child(lines2)